In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import requests
from bs4 import BeautifulSoup
import time
import seaborn as sns
# import folium

In [ ]:
data = pd.read_csv('data/houses/houses_current.csv')
data.head()
len(data)

In [ ]:
data.info()

REMOVE DUPLICATES

In [ ]:
print('Sum of duplicates: ', data.duplicated(subset=['houseTypes', 'area', 'num_floors', 'bedrooms','address','price']).sum())
data = data.drop_duplicates(subset=['houseTypes', 'area', 'num_floors', 'bedrooms','address','price']).copy()
print('After removing duplicates: ', data.duplicated().sum())
print('Length of data then: ', len(data))

CONVERT OBJECT TO FLOAT64 AND STRING

In [ ]:
data = data.astype({'houseTypes':'string','address':'string','url':'string'})

In [ ]:
data.info()

In [ ]:
for col in data.columns:
    print('Ratio of missing data of ' + str(col) +': ' + str(round(data[col].isna().sum()*100/len(data[col]),3)) + '%')

In [ ]:
data[['district','city']] = data['address'].apply(lambda x: pd.Series(str(x).split(', ')))


In [ ]:
data = data.drop(['address'], axis=True)

In [ ]:
data.reset_index(inplace=True, drop=True)

In [ ]:
data = data[['houseTypes','area','num_floors','bedrooms','district','city','price','url']]

In [ ]:
data.head()

In [ ]:
# data.to_csv('houses.csv',index=False)
# data.to_excel('houses.xlsx',index=False)

## Xử lí giá

In [ ]:
low_noise_data = data.loc[(data['price'] < 0.1)]
hign_noise_data = data.loc[(data['price'] >= 500)]
null_price_data = data[data['price'].isna()]

In [ ]:
house_data_1 = data.loc[(data['price'] >= 0.1) & (data['price'] < 10)]
fig,ax = plt.subplots(figsize = (8,5))
sns.distplot(house_data_1['price'])

In [ ]:
house_data_2 = data.loc[(data['price'] >= 0.1) & (data['price'] < 50)]
fig,ax = plt.subplots(figsize = (8,5))
sns.distplot(house_data_2['price'])

In [ ]:
house_data = data.loc[(data['price'] >= 0.1) & (data['price'] < 500)]

In [ ]:
#house_data.to_excel('house_data.xlsx', index=False)

In [ ]:
fig,ax = plt.subplots(figsize = (8,5))
sns.distplot(house_data['price'])

## Xử lí area

In [ ]:
house_data['area'].isna().sum()

In [ ]:
house_data['area'].describe()

In [ ]:
house_data = house_data.loc[(house_data['area'] >= 10) & (house_data['area'] <= 20000)]

In [ ]:
house_data['area'].describe()

In [ ]:
house_data_3 = house_data.loc[(house_data['area'] >=10) & (house_data['area'] <=500)]
fig,ax = plt.subplots(figsize = (8,5))
sns.distplot(house_data_3['area'])

# Xử lý num_floors và bedrooms

In [ ]:
house_data.isna().sum()

In [ ]:
house_data.loc[house_data['num_floors'] > 11, 'num_floors'] = None
house_data.isna().sum()

In [ ]:
fig,ax = plt.subplots(figsize = (8,5))
sns.distplot(house_data['num_floors'])

In [ ]:
fig,ax = plt.subplots(figsize = (8,5))
sns.distplot(house_data['bedrooms'])

In [ ]:
numeric_houses = pd.DataFrame({'area':house_data['area'],'num_floors':house_data['num_floors'],'bedrooms':house_data['bedrooms'],'price':house_data['price']})
numeric_houses.isna().sum()

In [ ]:
len(numeric_houses)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.impute import KNNImputer

rmse = lambda y, yhat: np.sqrt(mean_squared_error(y, yhat))

In [ ]:
def optimize_k(data, target):
    errors = []
    for k in range(1, 20):
        imputer = KNNImputer(n_neighbors=k)
        imputed = imputer.fit_transform(data)
        df_imputed = pd.DataFrame(imputed, columns=data.columns)
        
        X = df_imputed.drop(target, axis=1)
        y = df_imputed[target]
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

        model = RandomForestRegressor()
        model.fit(X_train, y_train)
        preds = model.predict(X_test)
        error = rmse(y_test, preds)
        errors.append({'K': k, 'RMSE': error})
        
    return errors

In [ ]:
test_numeric_house = numeric_houses.copy()
test_numeric_house.isna().sum()

In [ ]:
k_errors = optimize_k(data=test_numeric_house, target='price')
print(k_errors)

In [ ]:
imputer = KNNImputer(n_neighbors=17)
imputed = imputer.fit_transform(numeric_houses)
df_imputed = pd.DataFrame(imputed, columns=numeric_houses.columns)

In [ ]:
df_imputed.isna().sum()

In [ ]:
df_imputed['bedrooms'].describe()

In [ ]:
# numeric_houses.to_excel('numeric_houses.xlsx',index=False)
# df_imputed.to_excel('df_imputed.xlsx',index=False)

In [ ]:
house_data = house_data.reset_index()

In [ ]:
house_data['new_num_floors'] = df_imputed['num_floors'].round(0)
house_data['new_bedrooms'] = df_imputed['bedrooms'].round(0)

In [ ]:
house_data.isna().sum()

In [ ]:
# house_data.to_excel('full_houses.xlsx', index=False)
house_data

# Thử nghiệm KNN cho missing values

In [ ]:
not_null_houses = house_data.dropna() # phải lấy house_data lúc còn null (trước khi fill ở phần trên)
not_null_houses.to_excel('data/not_null_houses.xlsx',index=False)

In [ ]:
i1 = np.random.choice(a=not_null_houses.index, size=50)
i2 = np.random.choice(a=not_null_houses.index, size=50)
print(i1)
print(i2)

In [ ]:
null_houses = pd.DataFrame({'area':not_null_houses['area'],'num_floors':not_null_houses['num_floors'],'bedrooms':not_null_houses['bedrooms'],'price':not_null_houses['price']})
null_houses.isna().sum()

In [ ]:
null_houses.loc[i1, 'num_floors'] = np.nan
null_houses.loc[i2, 'bedrooms'] = np.nan

In [ ]:
#null_houses.loc[164]

In [ ]:
null_houses.isna().sum()

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.impute import KNNImputer

rmse = lambda y, yhat: np.sqrt(mean_squared_error(y, yhat))

In [ ]:
def optimize_k(data, target):
    errors = []
    for k in range(1, 20, 2):
        imputer = KNNImputer(n_neighbors=k)
        imputed = imputer.fit_transform(data)
        df_imputed = pd.DataFrame(imputed, columns=data.columns)
        
        X = df_imputed.drop(target, axis=1)
        y = df_imputed[target]
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

        model = RandomForestRegressor()
        model.fit(X_train, y_train)
        preds = model.predict(X_test)
        error = rmse(y_test, preds)
        errors.append({'K': k, 'RMSE': error})
        
    return errors

In [ ]:
k_errors = optimize_k(data=null_houses, target='price')
print(k_errors)

In [ ]:
imputer_test = KNNImputer(n_neighbors=13)
imputed_test = imputer_test.fit_transform(null_houses)
df_imputed_test = pd.DataFrame(imputed_test, columns=null_houses.columns)

In [ ]:
df_imputed_test.isna().sum()
df_imputed_test.to_excel('data/df_imputed_test.xlsx',index=False)

Note: so sánh not_null_houses.xlsx và df_imputed_test.xlsx

 # Initialize google map

In [ ]:
from geopy.geocoders import Nominatim

geolocator = Nominatim(user_agent='my_app')

# Tạo đối tượng geocoder
geocoder = geolocator.geocode

# Geocode the addresses to obtain latitude and longitude coordinates
locations = [geocoder(f"{district}, {city}") for district, city in zip(house_data['district'][0:100], house_data['city'][0:100])]

# Extract latitude and longitude coordinates from the locations
latitude = [location.latitude for location in locations]
longitude = [location.longitude for location in locations]

# Mark points on google map

In [ ]:
lat = 10.861410750000001
long = 106.66420925636413
zoom = 6
gmap2 = folium.Map(location=(lat, long), zoom_start=zoom)
def _addMarker(latitudes, longitudes,gmap):
    # Kiểm tra độ dài của danh sách latitude và longitude
    if len(latitudes) != len(longitudes):
        print("Danh sách latitude và longitude không cùng độ dài.")
        return
    # Thêm Marker vào map_obj
    for lat, lng in zip(latitudes, longitudes):
        marker = folium.Marker(location=(lat, lng))
        marker.add_to(gmap)
_addMarker(latitude,longitude,gmap2)
gmap2.save("foliumMarkerMultiple.html")

# Heatmap

In [ ]:
from folium.plugins import HeatMap

lat = 10.861410750000001
long = 106.66420925636413
zoom = 6
gmap2 = folium.Map(location=(lat, long), zoom_start=zoom)
maximum = max(house_data['price'])

def _addHeatMap(map_obj, latitudes, longitudes, values):
    # Tạo danh sách các tọa độ (latitude, longitude, value)
    data = list(zip(latitudes, longitudes, values))
    # Tạo HeatMap từ danh sách tọa độ và values
    heat_map = HeatMap(data)
    # Thêm HeatMap vào bản đồ
    map_obj.add_child(heat_map)
_addHeatMap(gmap2,latitude,longitude,house_data['price'][0:100])
gmap2.save("foliumHeatMap.html")

# Encoding

In [ ]:
new_house_data = pd.read_excel('data/full_houses.xlsx')
new_house_data.drop(['district','url','level_0','index','num_floors','bedrooms'],axis=1,inplace=True)
new_house_data = new_house_data[['houseTypes','area','new_num_floors','new_bedrooms','city','price']]
new_house_data

# OneHotEncoder

In [ ]:
from sklearn.preprocessing import OneHotEncoder
# Tạo một đối tượng OneHotEncoder
encoder = OneHotEncoder(sparse=False)

# Chọn cột cần mã hóa
columns_to_encode = ['houseTypes','city']

# Mã hóa one-hot cho các cột
encoded_data = encoder.fit_transform(new_house_data[columns_to_encode])


# Tạo DataFrame mới từ dữ liệu đã mã hóa
encoded_df = pd.DataFrame(encoded_data, columns=encoder.get_feature_names(columns_to_encode))

# Xóa các cột gốc trong biến data
new_house_data.drop(columns_to_encode, axis=1, inplace=True)

# Kết hợp dữ liệu đã mã hóa với biến data
new_house_data = pd.concat([new_house_data, encoded_df], axis=1)

# In ra 5 dòng đầu tiên của dữ liệu sau khi mã hóa
print(new_house_data.head())

In [ ]:
print(new_house_data.to_excel('oneHot.xlsx', index=False))

# Label Encoder

In [ ]:
new_house_data = pd.read_excel('data/new_house_data.xlsx')
new_house_data.drop(['district','url'],axis=1,inplace=True)

In [ ]:
from sklearn.preprocessing import LabelEncoder

# Tạo một đối tượng LabelEncoder
label_encoder = LabelEncoder()

# Mã hóa nhãn cho cột 'loại nhà'
new_house_data['houseTypes_encoded'] = label_encoder.fit_transform(new_house_data['houseTypes'])

# Mã hóa nhãn cho cột 'city'
new_house_data['city_encoded'] = label_encoder.fit_transform(new_house_data['city'])

new_house_data.drop(['city','houseTypes'],axis=1,inplace=True)

# In ra 5 dòng đầu tiên của dữ liệu đã mã hóa
print(new_house_data.head())

In [ ]:
print(new_house_data.to_excel('Label.xlsx', index=False))

# TargetEncoder for city and OneHot for housetypes 

In [ ]:
new_house_data = pd.read_excel('data/new_house_data.xlsx')
new_house_data.drop(['district','url'],axis=1,inplace=True)

In [ ]:
#pip install category_encoders

In [ ]:
import category_encoders as ce
from sklearn.preprocessing import OneHotEncoder

target_encoder =ce.TargetEncoder() 
df_city=target_encoder.fit_transform(new_house_data['city'],new_house_data['price'])
df_new = df_city.join(new_house_data.drop('city',axis = 1))

df_new['city_new'] = df_new[city]
# Tạo một đối tượng OneHotEncoder
encoder = OneHotEncoder(sparse=False)
# Chọn cột cần mã hóa
columns_to_encode = ['houseTypes']
# Mã hóa one-hot cho các cột
encoded_data = encoder.fit_transform(new_house_data[columns_to_encode])
# Tạo DataFrame mới từ dữ liệu đã mã hóa
encoded_df = pd.DataFrame(encoded_data, columns=encoder.get_feature_names_out(columns_to_encode))

# Xóa các cột gốc trong biến data
df_new.drop(columns_to_encode, axis=1, inplace=True)

# Kết hợp dữ liệu đã mã hóa với biến data
df_new = pd.concat([df_new, encoded_df], axis=1)

print(df_new.head())

In [ ]:
print(df_new.to_excel('data/houses/Target.xlsx', index=False))

# Derived features

### Standalize

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
features_remove_target = ['city_area', 'population', 'gdp']

In [ ]:

df_unit = df_city = pd.read_excel('data/city_features/city.xlsx')
df_tranform = pd.DataFrame(data = StandardScaler().fit_transform(df_unit.loc[:, features_remove_target ].values), columns = features_remove_target)


### PCA

In [ ]:
pca = PCA(n_components = 1)
component = pd.DataFrame(pca.fit_transform(df_tranform.values), columns = ['city_index'])
df_city_index = pd.concat([df_unit['city'], component], axis=1)
print(df_city_index)

In [ ]:
df_house = pd.read_excel("data/houses/Target.xlsx")

In [ ]:
new_df = pd.merge(df_house, df_city_index, on= 'city')
new_df = new_df[['price',\
        'city',\
        'city_target_encoder',\
        'city_index',\
        'area',\
        'new_num_floors',\
        'new_bedrooms',\
        'houseTypes_Bán Luxury home',\
        'houseTypes_Bán Nhà',\
        'houseTypes_Bán Nhà cổ',\
        'houseTypes_Bán Nhà mặt phố',\
        'houseTypes_Bán Nhà riêng']]

In [ ]:
features = ['price',\
        'city',\
        'city_target_encoder',\
        'city_index',\
        'area',\
        'new_num_floors',\
        'new_bedrooms',\
        'houseTypes_Bán Luxury home',\
        'houseTypes_Bán Nhà',\
        'houseTypes_Bán Nhà cổ',\
        'houseTypes_Bán Nhà mặt phố',\
        'houseTypes_Bán Nhà riêng']

In [ ]:
print(new_df.to_excel('final_data.xlsx', index=False))

In [ ]:
new_df.describe